## M343 - Applied Number Theory
## Assignment #2
### Abigail Johnson ( amj2694 )
### Fall 2016

In [2]:
from __future__ import division
from math import factorial
import sys
import os
import math
import numpy as np
import re
from sympy.ntheory import factorint

IN_FILE = 'input.txt'
OUT_FILE = 'output.txt'


# opens file specified by name, returns contents in array of strings
def process_file( file_name ):
    return open(file_name).read().splitlines()


# Returns gcd(x,y) using Euclidean algorithm
# Uses fact that (1) m = nq + r where 0 <= r < n ( for m,n,q,r in Z )
#                (2) gcd(m,n) = gcd(n,r)    
def gcd(x,y):
    # slight performance optimization ( x > 0 added to account neg num)
    if y > x and x > 0 :
        return gcd(y,x)
    elif x % y == 0:
        return abs(y)
    else:
        return abs(gcd( y , x % y ))
    
    
# Computes addition or multiplication (denoted by op) for x and y in ring m    
def addition_multiplication_ring(m, op, x, y):
    if op == "*":
        result = ( x * y) % m
    else:
        result = ( x + y) % m
    return result


# Decode Message Encrypted with Shift Cypher
def decode_shift_cipher(message):
    # 1. Preprocess message and setup dicts 
    message = message.lower()
    message = re.sub(r'[^a-zA-Z ]+', '', message)
    
    # 2. Set up dict objects used for processing 
    alph_dict = { 'a' : 0, 'b': 1, 'c': 2, 'd': 3, 'e' : 4, 'f' : 5, 'g' : 6, 'h' : 7, 'i' : 8, 
                 'j' : 9, 'k' : 10, 'l' : 11, 'm' : 12, 'n' : 13, 'o' : 14, 'p' : 15, 'q' : 16, 
                 'r' : 17, 's' : 18 , 't' : 19, 'u' : 20, 'v' : 21, 'w' : 22, 'x' : 23, 'y' : 24, 'z' : 25 }
    alph_dict_inv = { v: k for k, v in alph_dict.items() } # invert to allow reverse indexing 
    english_dict = enchant.Dict("en_US")
    shift_dict = {'key' : 0, 'valid_words' : 0, 'message': '' }
    
    # 2. Shift messasge 0-25 times, determine and record num valid words seen in shift_dict
    for key in range(0,26):            
        message_shift = [ char if char == ' ' else alph_dict_inv[ ( (alph_dict[char] + key) % 26 ) ] for char in message ]
        message_shift = ''.join(message_shift)
        valid_words = 0
        for word in message_shift.split(' '):
            if english_dict.check(word):
                valid_words += 1
        if valid_words >= shift_dict['valid_words']:
            shift_dict['key'] = key
            shift_dict['valid_words'] = valid_words
            shift_dict['message'] = message_shift
    return shift_dict


# computes g^x using fast exponentiation
def fast_exponentiation(g, x):
    if x == 0:
        return 1
    elif x % 2 == 1:
        return g * fast_exponentiation( g, x-1 )
    else:
        exp = fast_exponentiation( g, x/2)
        return exp * exp

    
# Computes a inverse mod p 
# using Euclidean Algorithm and fact that 
def inverse_mod_p( a, p ):
    inv = u_v_gcd(a,p)['u']
    return inv % p


# TODO: current does not handle b = 0 . fix it
# given a and b, returns gcd and u,v inc Integer st au + bv = gcd(a,b)
def u_v_gcd(a, b):
    u = 1; gcd = a; x = 0; y = b
    while True:
        if y == 0:
            v = ( gcd - ( a * u ) ) // b
            ret = {}
            ret['u'] = u ; ret['v'] = v ; ret['gcd'] = abs(gcd)
            return ret
        q = gcd // y 
        r = gcd % y
        s = u - ( q * x )
        u = x ; x = s; gcd = y ; y = r;
 

# Encrypts plaintext using elgamal encription
# returns ( c1, c2 )
# where c1 = ( g ^ k ) mod p
#       c2 = ( m * A ^ k ) mod p
#            where A = (g ^ a ) 
def elgamal_encryption(p, g, m, g_pow_a):
    y = np.random.randint( 1, p )
    print("y : ", y)
    c1 = fast_exponentiation( g_pow_a, y)
    c2 = c1 * inverse_mod_p( m, p)
    return ( c1, c2 )


# Input: A cyclic group G of order p, having a generator g and an element h.
# Output: A value x st g^x congr h modulo p
# Run-time : O( n^(1/2) )
def baby_step_giant_step(p, g, h):
    # 1. Compute limits
    m = math.ceil( math.sqrt(p) )   
    # 2. Giant Step
    giant_step = {}
    for i in range ( 0, m ):
        giant_step[ fast_exponentiation( g , i) % p ] = i    
    # 3. Baby Step / Intersection detection
    g_inv = inverse_mod_p( g, p )
    u = g_inv ** m 
    for i in range ( 0, m ):
        baby_step = ( ( u ** i ) * h ) % p 
        if baby_step in giant_step:
            x = m * i + giant_step[ baby_step ]
            return x
        
        
# TODO : implement as practice ( currently using preimplemented module )
# Returns prime factorization of given number
def prime_factorization(N):
    bases = []
    exponents = []   
    return bases, exponents 


# Uses chinese remaineder theorem to solve system of congruences of form:
#    x (output) = integers_i (mod modulus_i) for all i 
def chinese_remainder( a, n ):  
    #print("integers: ", a, " modulus: ", n)
    sum = 0
    prod = reduce(lambda a, b: a*b, n)
    for n_i, a_i in zip(n, a):
        p = prod / n_i
        inv = inverse_mod_p(p, n_i)
        #print('inverse_mod_p w/ p=', p," n=", n_i, " :\n", inv, "\n")
        sum += a_i * inv * p
        
    return sum % prod


# Pohlig-Hellman reduces discrete log for an element of order N to computing 
# discrete logs for elements of order q_i^{e_i}, for various primes q_i and exponents 
# e_i as specified by the prime factorization of N.
def pohlig_hellman(p, g, h ):
    N = p - 1
    # Calculate prime factorization 
    prime_fact = factorint(N)
    primes = [] ; exponents = []
    for key, value in prime_fact.items():
        primes.append(key)
        exponents.append(value)
    disc_logs = []
    modulus = [ fast_exponentiation( primes[i], exponents[i] ) for i in range( 0, len( primes ) ) ]
    # Calculate y_i for g_i ^ y_i = h_i mod p 
    for i in range( 0, len(primes) ):
        g_i = fast_exponentiation( g, ( N / modulus[i] ) )
        h_i = fast_exponentiation( h, ( N / modulus[i] ) )
        y_i = baby_step_giant_step( p , g_i, h_i )
        disc_logs.append( y_i % modulus[i] )  
    # Use Chinese remaineder theorem to piece together solution and solve
    # x = y_i ( mod q_i^ e_i ) for all i 
    return int( chinese_remainder(  disc_logs, modulus ) )

#### Problem 1 - Elgamal Encryption
Please write a computer program that performs Elgamal encryption. The input is a file “input.txt” that has p on the first line, g on the second line, m on the third line, and g^a (the value sent by Alice) on the fourth line. Output the result to “output.txt”.

In [3]:
def main():
    #print('Inverse mod p: ', inverse_mod_p(3589 , 32610))
    args = open('input.txt').read().splitlines()
    p = int(args[0]); g = int(args[1]) ; m = int(args[2]) ; g_pow_a = int(args[3]) 
    result = elgamal_encryption( p, g, m, g_pow_a )
    print( "Elgamal Encryption with p = ", p, ", g = ", g, ", m = ", m, ", g_pow_a = ", g_pow_a, " :\n \t e(m) = ", result)
    open('output.txt', 'w').write( str(result) )
    
if  __name__ =='__main__':
    main()

#### Problem 2 - Babystep Giantstep Algorithm
Write a computer program that implements the Babystep-Giantstep algorithm for solving the discrete log problem. The input is a file “in- put.txt” that has p on the first line, g on the second, and h on the third. Output the result to “output.txt”.

In [5]:
# Test file : p = 5, g = 2 , h = 1
def main():
    args = open('input.txt').read().splitlines()
    p = int(args[0]) ; g = int(args[1]) ; h = int(args[2])  
    result = baby_step_giant_step( p, g, h )
    print( "Result of Babystep Giantstep ( p = ", p, " g = ", g, " h = ", h, ") :\n ", result)
    open('output.txt', 'w').write( str(result) )
    
if  __name__ =='__main__':
    main()
    
#print(baby_step_giant_step(17,3,1))    
#print(baby_step_giant_step(31,3,6))

Result of Babystep Giantstep ( p =  2  g =  11250  h =  11250 ) :
  1


#### Problem 1.36
Compute 2^[ (p-1)/2 ] mod p for every prime 3 <= p < 20 and make conjectures about values. Prove your conjecture right.

In [40]:
primes = [ 3, 5, 7, 11, 13, 17, 19 ]
for p in primes:
    exp = ( p - 1 ) /2
    val =  ( fast_exponentiation(2, exp) ) % p
    
    print("2^[ (p-1)/2 ] mod p for p = ", p, " : ", val, " \n")
    

2^[ (p-1)/2 ] mod p for p =  3  :  2  

2^[ (p-1)/2 ] mod p for p =  5  :  4  

2^[ (p-1)/2 ] mod p for p =  7  :  1  

2^[ (p-1)/2 ] mod p for p =  11  :  10  

2^[ (p-1)/2 ] mod p for p =  13  :  12  

2^[ (p-1)/2 ] mod p for p =  17  :  1  

2^[ (p-1)/2 ] mod p for p =  19  :  18  



#### Discrete Log - Pohlig Hellman
For this problem, you will write an efficient discrete log solver, using the Shank’s solver you wrote for last homework as a base.
    (a) Implement the “shifting” technique for reducing discrete log mod p^n to discrete log mod p and arithmetic; the result should be a function which takes g, h, p, and n as inputs and solves g x = h mod p ^ n.
    (b) Implement the Pohlig-Hellman algorithm, using the function from the first part as your “black box” discrete log solver

In [57]:
# Test file : p = 5, g = 2 , h = 1
def main():
    args = open('input.txt').read().splitlines()
    p = int(args[0]) ; g = int(args[1]) ; h = int(args[2])  
    result = pohlig_hellman( p, g, h )
    print( "Result of Pohlig-Hellman ( p = ", p, " g = ", g, " h = ", h, ") :\n ", result)
    open('output.txt', 'w').write( str(result) )
    
if  __name__ =='__main__':
    main()
    
print(u_v_gcd(1291,-1074))
    

{'gcd': 1, 'u': -391, 'v': -470}


In [ ]:
# TODO: Add ouput to console stating : 
# is your input specified as follows ?
# if not enter order of variables seperated by commas,etc

# Future:
# Extact method from hw4
# Add console o/p of assumptions and/or assertions

### HW 4
1. Write program takes as i/p e, c, p and solves x^e = c mod p 

In [6]:
# Using proposition 3.2
# Let p be prime and e >= 1 integer satisfying 
#      gcd(e, p - 1) = 1
# Then there exists and integer d st        
#      de = 1 mod (p - 1).       
# Then the congruence 
#       x^e = c mod(p) 
# Has the unique solution
#       x = c^d mod(p)
def main():
    args = open('hw4_input.txt').read().splitlines()
    e = int(args[0]) ; c = int(args[1]) ; p = int(args[2])  
    d = inverse_mod_p( e, p - 1 )
    result = fast_exponentiation(c, d) % p
    print( "Given x^", e, "=", c, "mod(", p, "):\n \t x =", result)
    open('hw4_output.txt', 'w').write( str(result) )

if  __name__ =='__main__':
    main()
    
# 1st test file: e = 1, c = 2, p = 5 Expected Result: 2

Given x^ 1 = 2 mod( 5 ):
 	 x = 2


### HW 5
1: 
    1. i/p : p (prime) , m, q_1, q_2 inc Fp[t]
    2. o/p : q_1q_2 mod m and q_1 + q_2 mod m (ie computation takes place in quotient Fp[t]/(m)
2: 
    1. i/p : composite # N = pq
    2. o/p : p and q using Pollard's p - 1

In [4]:
from numpy.polynomial import polynomial as P

# Computes addition mod m for finite fields q1, q2
def field_add_mod_m(q1,q2, m,p):
    add = P.polyadd(q1,q2)
    #print("q1 + q2 = ", add)
    r, mod_add = P.polydiv(add,m)
    #print( "(NonSimplified) q1 + q2 mod m = ", mod_add )
    add_res = [ ( int(mod_add[i]) % (p)) for i in range(0, len(mod_add) ) ]
    #print("(Simplified) q1 + q2 mod m = ", add_res)
    return add_res

# Computes multiplication mod m for finite fields q1, q2
def field_mult_mod_m(q1,q2, m,p):
    mult = P.polymul(q1,q2)
    #print("q1 * q2 = ", mult)
    r, mod_mult = P.polydiv(mult,m)
    #print( "(NonSimplified) q1 * q2 mod m = ", mod_mult )
    mult_res = [ ( int(mod_mult[i]) % (p)) for i in range(0, len(mod_mult) ) ]
    #print("(Simplied) q1 * q2 mod m = " , mult_res)
    return mult_res

# TODO: what is point of p and m input? What do these look like
# 3 4 0 5 == 3 + 4x + 5x^3
def main():
    args = open('input.txt').read().splitlines()
    q1 = [ int(x) for x in args[0].split() ] 
    q2 = [ int(x) for x in args[1].split() ]
    m = [ int(x) for x in args[2].split() ]  
    p = int(args[3]) 
    res1 = field_mult_mod_m(q1,q2,m,p)
    res2 = field_add_mod_m(q1,q2,m,p)
    print( "Given q1 = ", q1, " q2 = ", q2, " m = ", m, ", p = ", p, " :\n ", 
          "\t q1*q2 = " , res1, " mod(", m ,") \n",
          "\t q1+q2 = " , res2, " mod(", m ,") \n" )
    open('output.txt', 'w').write( str(res1) + "\n" + str(res2) )

if  __name__ =='__main__':
    main()
    
"""
Input 1:
Given q1 =  [1, 0, 0, 0, 0]  q2 =  [1, 1, 1, 0, 0]  m =  [1, 1, 0, 1, 0] , p =  2  ) :
  	 q1*q2 =  [1, 1, 1]  mod( [1, 1, 0, 1, 0] ) correct
 	 q1+q2 =  [0, 1, 1]  mod( [1, 1, 0, 1, 0] ) 
Input 2:
Given q1 =  [1, 1, 0, 0, 0]  q2 =  [0, 0, 1, 0, 0]  m =  [1, 1, 0, 1, 0] , p =  2  ) :
  	 q1*q2 =  [1, 1, 1]  mod( [1, 1, 0, 1, 0] ) correct
 	 q1+q2 =  [1, 1, 1]  mod( [1, 1, 0, 1, 0] ) 
Input 3:
Given q1 =  [1, 1, 0, 0, 0]  q2 =  [1, 1, 0, 0, 0]  m =  [1, 1, 0, 1, 0] , p =  2  ) :
  	 q1*q2 =  [1, 0, 1]  mod( [1, 1, 0, 1, 0] ) correct
 	 q1+q2 =  [0, 0]  mod( [1, 1, 0, 1, 0] ) 
Input 4:
Given q1 =  [1, 0, 0, 0, 0]  q2 =  [0, 1, 0, 0, 0]  m =  [1, 1, 0, 1, 0] , p =  2  ) :
  	 q1*q2 =  [0, 1]  mod( [1, 1, 0, 1, 0] ) correct
 	 q1+q2 =  [1, 1]  mod( [1, 1, 0, 1, 0] ) 
Input 5:
Given q1 =  [1, 0, 0, 0, 0]  q2 =  [0, 1, 1, 0, 0]  m =  [1, 1, 0, 1, 0] , p =  2  ) :
  	 q1*q2 =  [0, 1, 1]  mod( [1, 1, 0, 1, 0] ) correct
 	 q1+q2 =  [1, 1, 1]  mod( [1, 1, 0, 1, 0] ) 
"""

Given q1 =  [1, 1, 0, 0, 0]  q2 =  [1, 1, 0, 0, 0]  m =  [1, 1, 0, 1, 0] , p =  2  :
  	 q1*q2 =  [1, 0, 1]  mod( [1, 1, 0, 1, 0] ) 
 	 q1+q2 =  [0, 0]  mod( [1, 1, 0, 1, 0] ) 



'\nInput 1:\nGiven q1 =  [1, 0, 0, 0, 0]  q2 =  [1, 1, 1, 0, 0]  m =  [1, 1, 0, 1, 0] , p =  2  ) :\n  \t q1*q2 =  [1, 1, 1]  mod( [1, 1, 0, 1, 0] ) correct\n \t q1+q2 =  [0, 1, 1]  mod( [1, 1, 0, 1, 0] ) \nInput 2:\nGiven q1 =  [1, 1, 0, 0, 0]  q2 =  [0, 0, 1, 0, 0]  m =  [1, 1, 0, 1, 0] , p =  2  ) :\n  \t q1*q2 =  [1, 1, 1]  mod( [1, 1, 0, 1, 0] ) correct\n \t q1+q2 =  [1, 1, 1]  mod( [1, 1, 0, 1, 0] ) \nInput 3:\nGiven q1 =  [1, 1, 0, 0, 0]  q2 =  [1, 1, 0, 0, 0]  m =  [1, 1, 0, 1, 0] , p =  2  ) :\n  \t q1*q2 =  [1, 0, 1]  mod( [1, 1, 0, 1, 0] ) correct\n \t q1+q2 =  [0, 0]  mod( [1, 1, 0, 1, 0] ) \nInput 4:\nGiven q1 =  [1, 0, 0, 0, 0]  q2 =  [0, 1, 0, 0, 0]  m =  [1, 1, 0, 1, 0] , p =  2  ) :\n  \t q1*q2 =  [0, 1]  mod( [1, 1, 0, 1, 0] ) correct\n \t q1+q2 =  [1, 1]  mod( [1, 1, 0, 1, 0] ) \nInput 5:\nGiven q1 =  [1, 0, 0, 0, 0]  q2 =  [0, 1, 1, 0, 0]  m =  [1, 1, 0, 1, 0] , p =  2  ) :\n  \t q1*q2 =  [0, 1, 1]  mod( [1, 1, 0, 1, 0] ) correct\n \t q1+q2 =  [1, 1, 1]  mod( [1, 1,

In [18]:
# Factors using pollards p - 1
import numpy as np

def pollard_fact(n):
    while True:
        a = np.random.randint(2,n-1)
        #if gcd(a,n) > 1:
        #    return a
        r = 2
        choose_new_bound = False
        while choose_new_bound == False:    
            # dynamic prodcut using : (a^(r−1)!)^r = a^r!
            a = fast_exponentiation( a, r )
            d = gcd( ( a - 1 ) ,n);
            #print( "d:",d1,"a-1:",a-1)
            if d == 1:
                choose_new_bound = True
            elif d < n: 
                return d
            else:
                r += 1
        
    return -1;
'''
def main():
    args = open('hw5-2_input.txt').read().splitlines()
    N = int(args[0]) 
    p = pollard_fact(N)
    q = int( N / p )
    print( "Using Pollard p - 1 ( N =" , N, ") : \n\t p = ", p, ", q = ", q )
    open('hw5-2_output.txt', 'w').write( str(p) + "\n" + str(q) )
    
if  __name__ =='__main__':
    main()
'''
# using 241 and 349    
# Solves the congruence x^e = c mod(p*q) given e, c, p, q 
# where p & q prime and gcd(p,q) = 1

'\ndef main():\n    args = open(\'hw5-2_input.txt\').read().splitlines()\n    N = int(args[0]) \n    p = pollard_fact(N)\n    q = int( N / p )\n    print( "Using Pollard p - 1 ( N =" , N, ") : \n\t p = ", p, ", q = ", q )\n    open(\'hw5-2_output.txt\', \'w\').write( str(p) + "\n" + str(q) )\n    \nif  __name__ ==\'__main__\':\n    main()\n'

### HW 6
1. Pollard p for discrete log
2. Implement RSA blind signature. Write program generate key pair for Alice so only i/p we need is message

In [3]:

def successor(x, s, t, p, g , h):
        N = (p - 1) / 2
        
        # Classify
        # 3n + 2 -> 0
        # 3n     -> 1
        # 3n + 1 -> 2
        clazz = ( x + 1 ) % 3
        if clazz == 0:
                return (  (h * x)  % p ) , ( (s + 1) % N ) , ( t ) 
        elif clazz == 1:
                return (  (x * x)  % p ) , ( (2 * s) % N ) , ( (2 * t) % N )
        else: # c == 2
                return (  (g * x)  % p ) , ( s ) , (  (t + 1) % N )

# Implements Pollard's Rho for solving Discreet Logs
# Runtime : O(log(N))
def pollards_rho(g, h, p):   
    N = (p - 1) / 2
    xa, sa, ta = 1, 0 ,0 
    xb, sb, tb = successor(1, 0, 0, p, g, h)
    while xa != xb:
        xa, sa, ta = successor(xa, sa, ta, p, g, h)
        # double step
        xb, sb, tb = successor(xb, sb, tb, p, g, h)
        xb, sb, tb = successor(xb, sb, tb, p, g, h)

    s, t = sa - sb % N , tb - ta % N
    if s == 0:
        print("ERROR pollard's rho")
        return -1
    elif s < 0:
        s +=  N
    if t < 0:
        t += N

    return  inverse_mod_p( s , N)  * t % N
    

def main():
    args = open('input.txt').read().splitlines()
    p = int(args[0]) ; g = int(args[1]) ; h = int(args[2])  
    result = pollards_rho( g, h, p )
    print( "Using Pollard's Rho, given ", g, "^x = ", h ," mod( ", p, " ): \n",
          "\t x = ", result )
    open('output.txt', 'w').write( str(int(result))  )

if  __name__ =='__main__':
    main()

Using Pollard's Rho, given  19 ^x =  29850  mod(  48611  ): 
 	 x =  8703.0


In [24]:
from random import randrange, getrandbits
from itertools import repeat
from functools import reduce

def message_to_integer(message):
    print("Converting message to integer...")
    print("\tMessage: ", message)
    encode = reduce(lambda x, y : (x << 8) + y, map(ord, message))
    print("\tEncoded message: ", encode)
    return encode

#Pick two large prime numbers, p and q. Let n=pq. Typically, n is a 1024 bit number. 
#Pick e relatively prime to (p-1)(q-1). Now find d such that ed=1 mod (p-1)(q-1). 
#The pair of numbers (N, e) is the public key. The pair of numbers (d, n) is the private key. 
#The two primes p,q are no longer needed, and can be discarded, but should never be revealed.
#The signature of message m is  s = md mod n.
def blind_RSA_sig( message ):
    # TODO: in practical application this would not be hardcoded
    p = 8703
    q = 48611
    
    N = ( p - 1 ) * ( q - 1 )
    e = pollard_fact(N)
    #print(e)
    # Find d such that ed=1 mod (p-1)(q-1)
    d = inverse_mod_p( N, e)
    
    blind_rsa = {}
    # signature of message m is  s = m^d mod n
    blind_rsa['signed'] = fast_exponentiation(message,d) % N
    # pair of numbers (N, e) is the public key
    blind_rsa['public_key'] = "(" + str(N) + "," + str(e) + ")"
    #print(blind_rsa)
    return blind_rsa

# Implement RSA blind signature. Write program generate key pair for Alice so only i/p we need is message
def main():
    args = open('input.txt').read().splitlines()
    message = args[0] 
    encoded_message = message_to_integer(message)
    rsa_sig = blind_RSA_sig(message)
    #result = pollards_rho()
    print( "Blind RSA signature. \n \t Signed message: ", rsa_sig['signed'] ,"\n\t Public Key:", rsa_sig['public_key'] )
    open('output.txt', 'w').write( rsa_sig['public_key'] + "/n" +  str(rsa_sig['signed']))
    
if  __name__ =='__main__':
    main()

Converting message to integer...
	Message:  1 1 0 0 0 
	Encoded message:  231989793353193238835232
76
Blind RSA signature. 
 	 Signed message:  1 
	 Public Key: (423004220,76)
